In [ ]:
#1A
import requests
from bs4 import BeautifulSoup

#Define URL
url = "https://realpython.github.io/fake-jobs/"

#GET request
response = requests.get(url)

#Convert to BS object
soup = BeautifulSoup(response.text, "html.parser")

#Use .find()
job_title_tag = soup.find("h2", class_="title is-5")

#Clean tags
job_title = job_title_tag.text.strip()

print("First Job Title:", job_title)

In [ ]:
#1B Find class "title is-5"
job_title_tags = soup.find_all("h2", class_="title is-5")

#Get text from each tag for list
job_titles = [tag.text.strip() for tag in job_title_tags]

print("Job Titles:", job_titles)

In [ ]:
#1C Get companies, locations, and dates

#Find all job cards
job_cards = soup.find_all("div", class_="card")

#Make list
companies = []
locations = []
dates = []

#Loop through each job card
for card in job_cards:
    
    #Get the company name class "subtitle is-6 company"
    company = card.find("h3", class_="subtitle is-6 company").text.strip()
    companies.append(company)
    
    #Get location
    location = card.find("p", class_="location").text.strip()
    locations.append(location)
    
    #Get date
    date = card.find("time").text.strip()
    dates.append(posting_date)

#Print
print("Companies:", companies)
print("Locations:", locations)
print("Dates:", dates)

In [ ]:
#1D Create a DataFrame
import pandas as pd

jobs_df = pd.DataFrame({
    "Job Title": job_titles,
    "Company": companies,
    "Location": locations,
    "Date": dates
})

jobs_df.head()

In [ ]:
#2A Get "Apply" URLs
apply_urls = []
for card in job_cards:
    # Find the anchor tag for "Apply"
    apply_link = card.find("a", string="Apply")
    # If found, get its href attribute; otherwise, append None
    if apply_link:
        apply_urls.append(apply_link.get("href"))
    else:
        apply_urls.append(None)

print("Apply URLs:", apply_urls)

#Add URLs as column in df
jobs_df["Apply URL"] = apply_urls
jobs_df.head()

In [ ]:
#2B Build the "Apply" URLs
import re

constructed_urls = []
base_url = "https://realpython.github.io/fake-jobs/jobs/"

for i, title in enumerate(job_titles):
    #Remove unneeded characters
    slug = re.sub(r'[^\w\s-]', '', title)
    #Make lowercase
    slug = slug.lower()
    #Replace extra whitespace characters with a -
    slug = re.sub(r'\s+', '-', slug)
    
    #Build the URL
    url_constructed = f"{base_url}{slug}-{i}.html"
    constructed_urls.append(url_constructed)

#Print URLs
print("Constructed URLs:", constructed_urls)

In [ ]:
#3A Get job page
job_url = "https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html"
job_response = requests.get(job_url)
job_soup = BeautifulSoup(job_response.text, "html.parser")

#Get job description
job_description_paragraph = job_soup.find("div", class_="content").find("p").text.strip()

print("Job Description:", job_description_paragraph)

In [ ]:
#3B Get all pages
def get_job_description(url):
    #Get the job page
    response = requests.get(url)
    #Parse the page
    soup = BeautifulSoup(response.text, "html.parser")
    # Find the container
    content_div = soup.find("div", class_="content")
    if content_div:
        description_p = content_div.find("p")
        if description_p:
            return description_p.text.strip()
    return ""

#Test function
test_url = "https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html"
job_description = get_job_description(test_url)
print("Job Description:", job_description)

In [ ]:
#3C Use .apply to get job descriptions
jobs_df["Description"] = jobs_df["Apply URL"].apply(get_job_description)

# Display results
jobs_df.head()

In [55]:
#Export to CSV
jobs_df.to_csv("jobs_data.csv", index=False)